<a href="https://colab.research.google.com/github/Leitich/Dynamic-Traffic-Controller/blob/main/Traffic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Object detection using a dataset of images and videos including cars, bikes, and trucks with TensorFlow:

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install tensorflow opencv-python matplotlib
!git clone https://github.com/tensorflow/models.git

In [ ]:
%cd /content/drive

/content/drive


In [ ]:
cd MyDrive/Traffic Data/models/research/object_detection


In [ ]:
!pip install -r requirements.txt


In [ ]:
!apt-get install protobuf-compiler
!pip install protobuf
import os
os.environ['PYTHONPATH'] += ':/content/models/research/object_detection'
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
import os
os.environ['PYTHONPATH'] += ':/content/models/research/object_detection:/content/models/research/object_detection/slim'


In [ ]:
!wget http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet50_coco_2018_01_28.tar.gz

--2022-12-16 06:18:24--  http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet50_coco_2018_01_28.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.141.128, 2607:f8b0:4023:c0b::80
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.141.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 381355771 (364M) [application/x-tar]
Saving to: ‘faster_rcnn_resnet50_coco_2018_01_28.tar.gz’

faster_rcnn_resnet5 100%[===================>] 363.69M  42.0MB/s    in 7.1s    

2022-12-16 06:18:31 (50.9 MB/s) - ‘faster_rcnn_resnet50_coco_2018_01_28.tar.gz’ saved [381355771/381355771]



In [ ]:
!tar -xvzf faster_rcnn_resnet50_coco_2018_01_28.tar.gz

faster_rcnn_resnet50_coco_2018_01_28/
faster_rcnn_resnet50_coco_2018_01_28/model.ckpt.index
faster_rcnn_resnet50_coco_2018_01_28/checkpoint
faster_rcnn_resnet50_coco_2018_01_28/pipeline.config
faster_rcnn_resnet50_coco_2018_01_28/model.ckpt.data-00000-of-00001
faster_rcnn_resnet50_coco_2018_01_28/model.ckpt.meta
faster_rcnn_resnet50_coco_2018_01_28/saved_model/
faster_rcnn_resnet50_coco_2018_01_28/saved_model/saved_model.pb
faster_rcnn_resnet50_coco_2018_01_28/saved_model/variables/
faster_rcnn_resnet50_coco_2018_01_28/frozen_inference_graph.pb


In [ ]:
cd ..

/content/drive/MyDrive/Traffic Data


In [ ]:
import os
import tensorflow as tf
from PIL import Image
from sklearn.preprocessing import MinMaxScaler
import tensorflow.keras as keras
import numpy as np
import keras.backend as K
import cv2
import matplotlib.pyplot as plt

In [ ]:
# Set the path to the model directory
model_dir = 'faster_rcnn_resnet101_coco_2018_01_28/saved_model/variables'

# Load the model
model = tf.saved_model.load(os.path.join(model_dir, 'saved_model'))

OSError: ignored

In [ ]:
!pwd

/content


In [ ]:
# Directory containing the images
image_dir = 'drive/MyDrive/Traffic Data/trainingdata'

# Get a list of files in the directory
image_names = os.listdir(image_dir)

# List of image file paths
image_paths = []

bboxes = []

# Loop over the file names
for image_name in image_names:
  # Create the full file path
  image_path = os.path.join(image_dir, image_name)
  
  # Add the file path to the list
  image_paths.append(image_path)

   # Load the image
  image = cv2.imread(image_path)
  
  # Detect objects in the image using an R-CNN model
  boxes, scores, classes, num = model.detect_objects(image)
  
  # Add the bounding box coordinates to the list
  bboxes.append(boxes)

# Create a dataset from the image file paths, labels, and bounding box coordinates
dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels, bboxes))


AttributeError: ignored

In [ ]:
# Load the labels for the images
labels = []

# Loop over the image file paths
for image_path in image_paths:
  # Extract the label from the file name
  label = image_path.split('_')[-1].split('.')[0]
  
  # Add the label to the list
  labels.append(label)

# Create a dataset from the image file paths and labels
dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))

In [ ]:
# Define a function to preprocess the images
def preprocess_image(image_path, label):
# Load the image and convert it to grayscale
  image = tf.io.read_file(image_path)
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.rgb_to_grayscale(image)
# Resize the image
  image = tf.image.resize(image, (200, 200))
# Normalize the image
  image = tf.cast(image, tf.float32) / 255.


In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(image_paths)
# Preprocess the images and convert them to a tuple of (image, label)
dataset = dataset.map(lambda image_path, label: (preprocess_image(image_path, lable), (image_path, label)))
# Shuffle the data and batch it
dataset = dataset.shuffle(1024).batch(32)

TypeError: ignored

In [ ]:
# Define the model
model = tf.keras.Sequential([
  # Add a convolutional layer
  tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(200, 200, 1)),
  # Add a pooling layer
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
  # Add another convolutional layer
  tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
  # Add another pooling layer
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
  # Flatten the output
  tf.keras.layers.Flatten(),
  # Add a dense layer
  tf.keras.layers.Dense(units=64, activation='relu'),
  # Add an output layer
  tf.keras.layers.Dense(units=1, activation='sigmoid')
])

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(dataset, epochs=10)